In [4]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
mp_drawing_styles = mp.solutions.drawing_styles

In [5]:
joint_list2 = [[8, 7, 6, 5], [12, 11, 10, 9], [16, 15, 14, 13], [20, 19, 18, 17]]

In [6]:
def decide_gesture(angle_list):
    threshold1 = 140
    threshold2 = 140
    if angle_list[0][1] > threshold1:
        if angle_list[1][1] > threshold1:
            if angle_list[2][1] > threshold1:
                if angle_list[3][1] > threshold1:
                    return 'four'
                else:
                    return 'three'
            else:
                return 'two'
        else:
            if angle_list[2][1] > threshold1:
                return 'one'
            else:
                if angle_list[3][1] > threshold1:
                    return 'rock'
                else:
                    return 'one'
    else:
        return 'none'

In [7]:
import math

def draw_and_get_finger_angles2(image, results, joint_list2):
    
    pos = 20
    RLmultiply = 0
    distance = None
    gesture = None
    # Loop through hands
    for RL, hand in enumerate(results.multi_hand_landmarks):
        angle_list = []
        try:
            if results.multi_handedness[RL].classification[0].label == 'Right':
                RLmultiply = 1
            else:
                RLmultiply = 0
        except:
            pass
        #Loop through joint sets 
        for num, joint in enumerate(joint_list2):
            a = np.array([hand.landmark[joint[0]].x, hand.landmark[joint[0]].y]) # First coord
            b = np.array([hand.landmark[joint[1]].x, hand.landmark[joint[1]].y]) # Second coord
            c = np.array([hand.landmark[joint[2]].x, hand.landmark[joint[2]].y]) # Third coord
            d = np.array([hand.landmark[joint[3]].x, hand.landmark[joint[3]].y]) # Fuorth coord
            e = np.array([hand.landmark[0].x, hand.landmark[0].y]) # Fifth coord
            
            radians1 = np.arctan2(b[1] - c[1], b[0]-c[0]) - np.arctan2(d[1]-c[1], d[0]-c[0])
            radians2 = np.arctan2(c[1] - d[1], c[0]-d[0]) - np.arctan2(e[1]-d[1], e[0]-d[0])
            angle1 = np.abs(radians1*180.0/np.pi)
            angle2 = np.abs(radians2*180.0/np.pi)
            
            if angle1 > 180.0:
                angle1 = 360-angle1
            if angle2 > 180.0:
                angle2 = 360-angle2
            
            angle_list.append((angle1, angle2))
                
            cv2.putText(image, str(round(angle1, 2)), (pos + num * 80 + RLmultiply * 320, 20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(round(angle2, 2)), (pos + num * 80 + RLmultiply * 320, 40),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
        if RLmultiply == 1:
            distance = math.sqrt((hand.landmark[4].x - hand.landmark[12].x)**2 + (hand.landmark[4].y - hand.landmark[12].y)**2)
            cv2.putText(image, str(round(distance, 2)), (int(hand.landmark[0].x * 640), int(hand.landmark[0].y * 480) + 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
        if RLmultiply == 0:
            gesture = decide_gesture(angle_list)
            cv2.putText(image, gesture, (int(hand.landmark[0].x * 640), int(hand.landmark[0].y * 480) + 30),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
    
    return image, distance, gesture

In [43]:
def draw_line(image,left_WRIST_y_coord):
    start_point = (0, left_WRIST_y_coord)
    end_point = (640, left_WRIST_y_coord)
    color = (0, 255, 0)
    thickness = 2
    image = cv2.line(image, start_point, end_point, color, thickness)
    return image

In [68]:
chord_mode_0 = {
    'one':"C",
    'two':"A",
    'three':"D",
    'four':"G"
}
chord_mode_1 = {
    'one':"1",
    'two':"2",
    'three':"3",
    'four':"4"
}
chord_modes = [num2chord_1, num2chord_2]

In [71]:
def change_chord_mode(chord_mode):
    num_of_mode = len(chord_modes)-1
    if chord_mode+1 <= num_of_mode:
        return chord_mode + 1
    else:
        return 0

In [69]:
def find_WRIST_coord(image, results):
    left_WRIST_y_coord = None
    right_WRIST_y_coord = None
    for RL, hand in enumerate(results.multi_hand_landmarks):
        if results.multi_handedness[RL].classification[0].label == 'Left':
            
            left_WRIST_y_coord = int(hand.landmark[mp_hands.HandLandmark.WRIST].y * 480)
            image = draw_line(image,left_WRIST_y_coord)
            
        else:
            right_WRIST_y_coord = int(hand.landmark[mp_hands.HandLandmark.WRIST].y * 480)
        
    return image, left_WRIST_y_coord, right_WRIST_y_coord

In [125]:
cap = cv2.VideoCapture(0)
counter = 0 # 計算偵數，讓模式間隔 N 偵以上才能切換
chord_mode = 0
N = 50
threshold = 0.2 
Status = True

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()
        
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )
                image, left_WRIST_y_coord, right_WRIST_y_coord = find_WRIST_coord(image, results)
                #print("left_WRIST_y_coord: ",left_WRIST_y_coord,"right_WRIST_y_coord: ",right_WRIST_y_coord,)
        
            # Draw angles to image from joint list
            image, distance, gesture = draw_and_get_finger_angles2(image, results, joint_list2)
            
        # Save our image    
        #cv2.imwrite(os.path.join('Output Images', '{}.jpg'.format(uuid.uuid1())), image)
        
        try:
            if right_WRIST_y_coord < left_WRIST_y_coord:
                Status = True          

            if gesture and right_WRIST_y_coord > left_WRIST_y_coord and Status: 
                play_chord(chord_modes[chord_mode][gesture])
                counter = 0
                Status = False

#             if distance > threshold:
#                 Status = True
#             if gesture and distance < threshold and counter > N and Status: # counter>20 讓音樂間隔 N 偵以上播放
#                 play_chord(num2chord[gesture])
#                 counter = 0
#                 Status = False
        except:
            try:
                if gesture == 'rock' and counter > N : # counter>20 讓模式間隔 N 偵以上才能切換
                    chord_mode = change_chord_mode(chord_mode)
                    print("change chord mode!")
                    counter = 0
                elif gesture == 'rock':
                    text = 'CHMOD'
                    cv2.putText(image, text, (150, 200), cv2.FONT_HERSHEY_SIMPLEX,
                      3, (0, 255, 255), 9, cv2.LINE_AA)
            except:
                pass
        
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
        counter = counter +1 

cap.release()
cv2.destroyAllWindows()

change chord mode!
change chord mode!
Thread G



    Error 263 for command:
        close music/G_chord.mp3
    指定的裝置未開啟，或無法由 MCI 所辨認。
Failed to close the file: music/G_chord.mp3


change chord mode!
change chord mode!
Thread C


In [90]:
gesture

'rock'

In [92]:
if gesture == 'rock':
    print("yeeee") 

yeeee


In [91]:
gesture == 'rock'

True

In [84]:
chord_mode = change_chord_mode(chord_mode)
chord_mode

1

In [56]:
from playsound import playsound
import threading
import time

def init_play_chord(chord):
    print("Thread", chord)
    playsound('music/{}_chord.mp3'.format(chord)) # 讓初次載入時發出聲音
    
def play_chord(chord):
    threads[chords[chord]] = threading.Thread(target = init_play_chord, args = (chord,))
    threads[chords[chord]].start()

chords = {
    "C": 0,
    "A": 1,
    "D": 2,
    "G": 3,
    "1": 0,
    "2": 1,
    "3": 2,
    "4": 3
}

# 建立 4 個子執行緒
threads = []
for i, chord_name in enumerate(chords):
    threads.append(threading.Thread(target = init_play_chord, args = (chord_name,)))
    threads[i].start()
    time.sleep(2)
    
for i in range(4):
    threads[i].join()

Thread C
Thread A
Thread D
Thread G
Thread 1
Thread 2
Thread 3
Thread 4


In [137]:
cap = cv2.VideoCapture(0)
counter = 0 # 計算偵數，讓模式間隔 N 偵以上才能切換
chord_mode = 0
N = 50
threshold = 0.2 
Status = True

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5) as hands: 
    while cap.isOpened():
        ret, frame = cap.read()
        
        
        # BGR 2 RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip on horizontal
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True
        
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )
                image, left_WRIST_y_coord, right_WRIST_y_coord = find_WRIST_coord(image, results)
                #print("left_WRIST_y_coord: ",left_WRIST_y_coord,"right_WRIST_y_coord: ",right_WRIST_y_coord,)
        
            # Draw angles to image from joint list
            image, distance, gesture = draw_and_get_finger_angles2(image, results, joint_list2)
            
        # Save our image    
        #cv2.imwrite(os.path.join('Output Images', '{}.jpg'.format(uuid.uuid1())), image)
        
        try:
            if right_WRIST_y_coord < left_WRIST_y_coord:
                Status = True          

            if gesture and right_WRIST_y_coord > left_WRIST_y_coord and Status: 
                play_chord(chord_modes[chord_mode][gesture])
                counter = 0
                Status = False

        except:
            try:
                if gesture == 'rock' and counter > N : # counter>20 讓模式間隔 N 偵以上才能切換
                    chord_mode = change_chord_mode(chord_mode)
                    print("change chord mode!")
                    counter = 0
                elif gesture == 'rock':
                    text = 'CHMOD'
                    cv2.putText(image, text, (0, 100), cv2.FONT_HERSHEY_SIMPLEX,
                      3, (0, 255, 255), 9, cv2.LINE_AA)
            except:
                pass
            
        cv2.putText(image, text, (470, 450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('Hand Tracking', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
            
        counter = counter +1 

cap.release()
cv2.destroyAllWindows()